In [ ]:
%matplotlib inline


# Estimate a probability with Monte Carlo


In this example we estimate a probability by means of a simulation algorithm, the Monte-Carlo algorithm. To do this, we need the classes `MonteCarloExperiment` and `ProbabilitySimulationAlgorithm`.



## Introduction

We consider a simple beam stressed by a traction load F at both sides.

![Axial stressed beam](axial-stressed-beam.png)

The geometry is supposed to be deterministic; the diameter D is equal to:

\begin{align}D=0.02 \textrm{ (m)}.\end{align}


By definition, the yield stress is the load divided by the surface. Since the surface is $\pi D^2/4$, the stress is:

\begin{align}S = \frac{F}{\pi D^2/4}.\end{align}


Failure occurs when the beam plastifies, i.e. when the axial stress gets larger than the yield stress:

\begin{align}R - \frac{F}{\pi D^2/4} \leq 0\end{align}


where $R$ is the strength.

Therefore, the limit state function $G$ is: 

\begin{align}G(R,F) = R - \frac{F}{\pi D^2/4},\end{align}


for any $R,F\in\mathbb{R}$.

The value of the parameter $D$ is such that:

\begin{align}D^2/4 = 10^{-4},\end{align}


which leads to the equation:

\begin{align}G(R,F) = R - \frac{F}{10^{-4} \pi}.\end{align}


We consider the following distribution functions.

 ========  ===========================================================================
 Variable  Distribution 
 ========  ===========================================================================
  R         LogNormal($\mu_R=3\times 10^6$, $\sigma_R=3\times 10^5$) [Pa]
  F         Normal($\mu_F=750$, $\sigma_F=50$) [N]
 ========  ===========================================================================

where $\mu_R=E(R)$ and $\sigma_R^2=V(R)$ are the mean and the variance of $R$.

The failure probability is: 

\begin{align}P_f = \text{Prob}(G(R,F) \leq 0).\end{align}


The exact $P_f$ is 

\begin{align}P_f = 0.02920.\end{align}




In [ ]:
from __future__ import print_function
import openturns as ot
import openturns.viewer as viewer
from matplotlib import pylab as plt
ot.Log.Show(ot.Log.NONE)

Create the joint distribution of the parameters.



In [ ]:
distribution_R = ot.LogNormalMuSigma(300.0, 30.0, 0.0).getDistribution()
distribution_F = ot.Normal(75e3, 5e3)
marginals = [distribution_R, distribution_F]
distribution = ot.ComposedDistribution(marginals)

Create the model.



In [ ]:
model = ot.SymbolicFunction(['R', 'F'], ['R-F/(pi_*100.0)'])

Create the event whose probability we want to estimate.



In [ ]:
vect = ot.RandomVector(distribution)
G = ot.CompositeRandomVector(model, vect)
event = ot.ThresholdEvent(G, ot.Less(), 0.0)

Create a Monte Carlo algorithm.



In [ ]:
experiment = ot.MonteCarloExperiment()
algo = ot.ProbabilitySimulationAlgorithm(event, experiment)
algo.setMaximumCoefficientOfVariation(0.05)
algo.setMaximumOuterSampling(int(1e5))
algo.run()

Retrieve results.



In [ ]:
result = algo.getResult()
probability = result.getProbabilityEstimate()
print('Pf=', probability)